# Modelagem Matemática com o JuMP

O JuMP é uma linguagem de modelagem matemática dentro do Julia.

Linguagens de modelagem recebem uma entrada do usuário descrevendo o problema de uma maneira agradável, e traduz para o **solver**, que irá resolver o problema. Existem diversos solvers, para diferentes tipos de problema.
Utilizaremos os seguintes:

- Cbc e Clp para programação linear e inteira
- Ipopt para programação não-linear

## Básicos

A ideia básica de criar um modelo em JuMP é definir
- Variáveis
- Objetivo
- Restrições

Tradicionalmente, utilizamos conjuntos de índices, que podem ou não ser utilizados em JuMP.

Vamos mostrar o básico do JuMP com alguns modelos simples.

### Exemplo simples

Vamos simplesmente encontrar a solução do problema
\begin{align*}
\max & \quad 2x + 3y \\
\text{suj. a} & \quad x + y \leq 3 \\
& \quad 3x + y \leq 5 \\
& \quad x + 2y \leq 4 \\
& \quad x, y \geq 0.
\end{align*}

In [ ]:
using Plots
gr(size=(600,400))

c = [2.0; 3.0]
contour(linspace(-0.1, 2.5, 30), linspace(-0.1, 2.5, 30),
        (x,y)->c[1]*x+c[2]*y, levels=20, leg=false,
        aspect_ratio=:equal)
plot!(x -> 2.2 - x, -0.2, 2.5, c=:black, lw=2)
plot!(x -> 5 - 3x, -0.2, 1.9, c=:black, lw=2)
plot!(x -> (4 - x) / 2, -0.2, 4.2, c=:black, lw=2)
plot!([0.0; 0.0], [0.0; 2.5], c=:black, lw=2)
plot!([0.0; 2.5], [0.0; 0.0], c=:black, lw=2)
plot!([0.0; c[1]/5], [0.0; c[2]/5], c=:red, l=:arrow)
xlims!(-0.1, 2.5)
ylims!(-0.1, 2.5)

In [ ]:
using JuMP, Clp

model = Model(solver = ClpSolver()) # Criar o modelo
@variable(model, x >= 0)
@variable(model, y >= 0)
@objective(model, Max, 2x + 3y)
@constraint(model, x + y <= 3)
@constraint(model, 3x + y <= 5)
@constraint(model, x + 2y <= 4)
status = solve(model)

println("status = $status")
println("x = $(getvalue(x))")
println("y = $(getvalue(y))")
println("f = $(getobjectivevalue(model))")

### Exemplo - problema de transporte

Uma empresa tem vários centros de produção e vários clientes. Quando um pedido é feito, é necessário decidir de onde sai o produto.

Vamos considerar um modelo simples onde cada centro de produção produz $a_i$ unidades, $i = 1,\dots,m$, e cada cliente tem uma demanda de $b_j$ unidades, $j = 1,\dots,n$.

Vamos denotar por $x_{i,j}$ a quantidade de produtos enviada do centro $i$ para o cliente $j$, e o custo de transporte por unidade é $c_{i,j}$.

Nosso objetivo é minimizar o custo, ou seja
$$ \min \sum_{i=1}^m \sum_{j=1}^n c_{i,j} x_{i,j}. $$

Para atender o cliente $j$, podemos enviar de qualquer centro de distribuição, ou seja
$$ \sum_{i=1}^m x_{i,j} \geq b_j, \quad \forall j = 1,\dots,n.$$

Para satisfazer a restrição de produção dos centros, a quantidade enviada para todos os clientes é limitada pelo produzido no centro. Daí,
$$ \sum_{j = 1}^n x_{i,j} \leq a_i, \quad \forall i = 1,\dots,m $$

Logo, nosso modelo é

In [ ]:
a = 10 * ones(5)
b = 5 * ones(10)
m, n = length(a), length(b)
C = rand(10:10:90, m, n) # Fácil de visualizar, mas sem significado físico

In [ ]:
# Vamos fazer o modelo mais geral
model = Model(solver = ClpSolver())
@variable(model, x[1:m,1:n] >= 0) # x tem dois índices
@objective(model, Min, sum(C[i,j] * x[i,j] for i = 1:m, j = 1:n))
@constraint(model,
            demanda[j=1:n], # Este demanda é um label, e [j=1:n] faz um tipo de for no j
            sum(x[i,j] for i = 1:m) >= b[j])
@constraint(model, restricao[i=1:m], sum(x[i,j] for j = 1:n) <= a[i])
print(model)

In [ ]:
status = solve(model)
x = getvalue(x)
fx = getobjectivevalue(model)

for i = 1:m
    for j = 1:n
        if x[i,j] > 0
            println("Envia $(x[i,j]) de $i a $j")
        end
    end
end

O JuMP cria internamente a matriz de coeficientes. Não vale a pena tentar usá-la. O solver faz isso.

In [ ]:
A = MathProgBase.getconstrmatrix(model.internalModel)
full(A)

In [ ]:
λdemanda = getdual(demanda)

In [ ]:
λrestricao = getdual(restricao)

### Modelo não-linear

Para problemas não-lineares gerais, só temos um solver livre para usar, o Ipopt.

\begin{align*}
\min & \quad \sum_i x_i^2 \\
\text{suj. a} & \quad x_1^2 + 4x_2^2 \geq 1.
\end{align*}

In [ ]:
using Ipopt

model = Model(solver = IpoptSolver())
@variable(model, x[1:2], start=1.0)
@NLobjective(model, Min, sum(x[i]^2 for i = 1:2))
@NLconstraint(model, x[1]^2 + 4 * x[2]^2 >= 1)
status = solve(model)

x = getvalue(x)
fx = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("f = $fx")

In [ ]:
contour(linspace(-2, 2, 100), linspace(-2, 2, 100), (x,y)->x^2 + y^2 ,leg=false)
t = linspace(0, 2π, 360)
plot!(cos.(t), 0.5sin.(t), c=:blue, lw=2)

scatter!([x[1]], [x[2]], ms=5, c=:red)

\begin{align*}
\min & \quad \sum_i x_i^2 \\
\text{suj. a} & \quad (x_1 - \tfrac{1}{2})^2 + 4(x_2 - \tfrac{1}{2})^2 \geq 2.
\end{align*}

In [ ]:
using Ipopt

model = Model(solver = IpoptSolver())
@variable(model, x[i=1:2], start=2.0) # Starting point: [2; 2]
@NLobjective(model, Min, sum(x[i]^2 for i = 1:2))
@NLconstraint(model, (x[1] - 0.5)^2 + 4 * (x[2] - 0.5)^2 >= 2) # (x₁ - ½)² + 4 (x₂ - ½)² ≥ 2
status = solve(model)

x = getvalue(x)
fx = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("f = $fx")

In [ ]:
contour(linspace(-2, 2, 100), linspace(-2, 2, 100), (x,y)->x^2 + y^2 ,leg=false)
t = linspace(0, 2π, 360)
plot!(sqrt(2) * cos.(t) + 0.5, (1/sqrt(2))*sin.(t) + 0.5, c=:blue, lw=2)

scatter!([x[1]], [x[2]], ms=5, c=:red)

In [ ]:
using Ipopt

model = Model(solver = IpoptSolver())
@variable(model, x[i=1:2], start=0.0)
@NLobjective(model, Min, sum(x[i]^2 for i = 1:2))
@NLconstraint(model, (x[1] - 0.5)^2 + 4 * (x[2] - 0.5)^2 >= 2)
status = solve(model)

x = getvalue(x)
fx = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("f = $fx")

In [ ]:
contour(linspace(-2, 2, 100), linspace(-2, 2, 100), (x,y)->x^2 + y^2 ,leg=false)
t = linspace(0, 2π, 360)
plot!(sqrt(2) * cos.(t) + 0.5, (1/sqrt(2))*sin.(t) + 0.5, c=:blue, lw=2)

scatter!([x[1]], [x[2]], ms=5, c=:red)

*Parametrizando a elipse*

In [ ]:
xel(t) = sqrt(2) * cos(t) + 0.5
yel(t) = (1/sqrt(2)) * sin(t) + 0.5
ϕ(t) = xel(t)^2 + yel(t)^2

t = linspace(0, 2π, 100)
plot(t, ϕ.(t), lw=2, c=:blue)

### Caixa de leite

Uma caixa de leite pode ser planificada como duas folhas de dimensões $W$ e $H$.
Considere uma folha dessas. À uma distância $x$ de cada parede da caixa, está a posição onde a folha foi dobrada para montar uma caixa.
Queremos minimizar o custo de material (folha) mantendo o volume da caixa.

In [ ]:
w, h = 5, 7
x = 1
plot([0; w; w; 0; 0], [0; 0; h; h; 0], c=:black, leg=false, aspect_ratio=:equal)
plot!([0; x], [0; x], c=:black, l=:dash)
plot!([0; x], [h; h - x], c=:black, l=:dash)
plot!([w; w - x], [0; x], c=:black, l=:dash)
plot!([w; w - x], [h; h - x], c=:black, l=:dash)
plot!([x; w-x; w-x; x; x], [x; x; h-x; h-x; x], c=:black, l=:dash)
xlims!(-1, w + 1)
ylims!(-1, h + 1)

In [ ]:
w, h = 5, 7
x = 1
P = [  x   x  x;
     w-x   x  x;
     w-x h-x  x;
       x h-x  x;
       x   x -x;
     w-x   x -x;
     w-x h-x -x;
     #  x h-x -x
    ]
np = size(P, 1)
plot(leg=false)
for i = 1:np
    for j = i+1:np
        p1, p2 = P[i,:], P[j,:]
        if count(p1 - p2 .== 0) == 2
            plot!([p1[1], p2[1]], [p1[2], p2[2]], [p1[3], p2[3]], c=:black)
        end
    end
end
#plot!(x * ones(3), [x; 2x; x], [x; 0; -x], c=:red, l=:dash)
plot!((w - x) * ones(3), [x; 2x; x], [x; 0; -x], c=:red, l=:dash)
#plot!(x * ones(3), [h-x; h-2x; h-x], [x; 0; -x], c=:red, l=:dash)
plot!((w - x) * ones(3), [h-x; h-2x; h-x], [x; 0; -x], c=:red, l=:dash)
plot!([x; w-x; w-x], [x; x; h-x], zeros(3), c=:blue, l=:dot)
xlims!(0, w)
ylims!(0, h)
zlims!(-2x, 2x)

Pelas figuras, vemos que o volume é $2x \times (W - 2x) \times (H - 2x)$, e que a quantidade de material é $W \times H$.

In [ ]:
volume = 100

model = Model(solver = IpoptSolver(print_level=0))
@variables model begin
    W >= 0
    H >= 0
    x >= 0
end
@objective(model, Min, W * H)
@NLconstraint(model, 2x * (W - 2x) * (H - 2x) == volume)
@constraint(model, 2x <= H)
@constraint(model, 2x <= W)

status = solve(model)
w, h, x = getvalue(W), getvalue(H), getvalue(x)
println("W = $w, H = $h, x = $x")

In [ ]:
plot([0; w; w; 0; 0], [0; 0; h; h; 0], c=:black, leg=false, aspect_ratio=:equal)
plot!([0; x], [0; x], c=:black, l=:dash)
plot!([0; x], [h; h - x], c=:black, l=:dash)
plot!([w; w - x], [0; x], c=:black, l=:dash)
plot!([w; w - x], [h; h - x], c=:black, l=:dash)
plot!([x; w-x; w-x; x; x], [x; x; h-x; h-x; x], c=:black, l=:dash)
xlims!(-1, w + 1)
ylims!(-1, h + 1)

In [ ]:
P = [  x   x  x;
     w-x   x  x;
     w-x h-x  x;
       x h-x  x;
       x   x -x;
     w-x   x -x;
     w-x h-x -x;
     #  x h-x -x
    ]
np = size(P, 1)
plot(leg=false)
for i = 1:np
    for j = i+1:np
        p1, p2 = P[i,:], P[j,:]
        if count(p1 - p2 .== 0) == 2
            plot!([p1[1], p2[1]], [p1[2], p2[2]], [p1[3], p2[3]], c=:black)
        end
    end
end
#plot!(x * ones(3), [x; 2x; x], [x; 0; -x], c=:red, l=:dash)
plot!((w - x) * ones(3), [x; 2x; x], [x; 0; -x], c=:red, l=:dash)
#plot!(x * ones(3), [h-x; h-2x; h-x], [x; 0; -x], c=:red, l=:dash)
plot!((w - x) * ones(3), [h-x; h-2x; h-x], [x; 0; -x], c=:red, l=:dash)
plot!([x; w-x; w-x], [x; x; h-x], zeros(3), c=:blue, l=:dot)
xlims!(0, w)
ylims!(0, h)
zlims!(-2x, 2x)

### Empacotamento de círculos

Quero empacotar $n$ círculos de raio $r$ no menor raio possível. Podemos modelar como encontrar o raio $R$ de um círculo maior centrado na origem, tal que todos círculos pequenos estejam dentro, minimizando o raio $R$.

Cada $(x_i,y_i)$ corresponde ao centro de um círculo pequeno. Visualmente, é fácil ver que para um círculo menos esteja completamente dentro do círculo maior, seu centro deve estar no máximo à uma distância $R - r$.
Para que não haja colisão, dois centros devem estar à uma distância $2r$.
Colocando no modelo, elevando ao quadrado quando for possível, temos:

\begin{align*}
\min & \quad R \\
\text{suj. a} & \quad x_i^2 + y_i^2 \leq (R - r)^2,
    \quad \forall i = 1,\dots,n \\
& \quad (x_i - x_j)^2 + (y_i - y_j)^2 \geq 4r^2
    \quad \forall i, j = 1,\dots,n, \ i \neq j.
\end{align*}

In [ ]:
using Ipopt
gr(size=(400,400))

n = 5
r = 1.0

model = Model(solver = IpoptSolver(print_level=0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=0.0)
@variable(model, y[i=1:n], start=0.0)
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
fx = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("f = $fx")

Nossa tentativa acima falhou. O Ipopt retorna com problema de restauração (correção). Isso quer dizer que não conseguiu achar um ponto factível. Diferentemente dos problemas de PL, em PNL não existe nem a garantia de que encontraremos uma solução factível. *Vamos tentar melhorar o desempenho do método colocando uma solução inicial*.

In [ ]:
t = linspace(0, 2π, 360)
plot(leg=false)
for i = 1:n
    plot!(r * cos.(t) + n * cos(2π * i / n), r * sin.(t) + n * sin(2π * i / n), c=:blue)
end
plot!()

In [ ]:
using Ipopt

n = 19
r = 1.0

model = Model(solver = IpoptSolver(print_level=0, max_cpu_time=5.0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=n * cos(2π * i / n))
@variable(model, y[i=1:n], start=n * sin(2π * i / n))
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
R = getvalue(R)
f = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("R = $R")
println("f = $f")

In [ ]:
t = linspace(0, 2π, 360)
plot(R * cos.(t), R * sin.(t), lw=2, c=:green, leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

# Volte e faça para n = 19

Infelizmente, nossa solução inicial não ficou boa. Vamos tentar outra.

In [ ]:
t = linspace(0, 2π, 360)
plot(leg=false)
for i = 1:n
    plot!(r * cos.(t) + cos(4π * i / (2n + 1)), r * sin.(t) + sin(4π * i / (2n + 1)), c=:blue)
end
plot!()

In [ ]:
using Ipopt

n = 19
r = 1.0

model = Model(solver = IpoptSolver(print_level=0, max_cpu_time=5.0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=cos(4π * i / (2n + 1)))
@variable(model, y[i=1:n], start=sin(4π * i / (2n + 1)))
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
R = getvalue(R)
f = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("R = $R")
println("f = $f")

In [ ]:
t = linspace(0, 2π, 360)
plot(R * cos.(t), R * sin.(t), lw=2, c=:green, leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

Apesar da solução inicial anterior ser factível, era muito próxima de um minimizador local. A última não era próxima de minimizador local, e por sorte permitiu achar uma solução melhor, mesmo sendo infactível.

In [ ]:
using Ipopt

n = 19
r = 1.0

model = Model(solver = IpoptSolver(print_level=0, max_cpu_time=5.0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=rand())
@variable(model, y[i=1:n], start=rand())
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
R = getvalue(R)
f = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("R = $R")
println("f = $f")

In [ ]:
t = linspace(0, 2π, 360)
plot(R * cos.(t), R * sin.(t), lw=2, c=:green, leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

# Tempo

In [ ]:
using Ipopt

n = 38
r = 1.0

model = Model(solver = IpoptSolver(print_level=0, max_cpu_time=30.0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=rand())
@variable(model, y[i=1:n], start=rand())
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
R = getvalue(R)
f = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("R = $R")
println("f = $f")

In [ ]:
t = linspace(0, 2π, 360)
plot(R * cos.(t), R * sin.(t), lw=2, c=:green, leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

### Solução inicial decente

In [ ]:
function initial_packing(n, r)
    x, y = zeros(n), zeros(n)
    lvl = 1
    i = 2
    while i <= n
        θ = 0.0
        x[i] = 2r * lvl
        y[i] = 0.0
        δ = acos(1 - 1 / (2 * lvl^2))
        while θ <= 2π - 1.999δ && i < n
            θ += δ
            i += 1
            x[i] = 2lvl * r * cos(θ)
            y[i] = 2lvl * r * sin(θ)
        end
        i += 1
        lvl += 1
    end
    return x, y
end

In [ ]:
n = 19
x, y = initial_packing(n, 1.0)
t = linspace(0, 2π, 360)
plot(leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

In [ ]:
using Ipopt

n = 72
r = 1.0

x0, y0 = initial_packing(n, 1.0)
model = Model(solver = IpoptSolver(print_level=0, max_cpu_time=30.0))
@variable(model, R >= r)
@variable(model, x[i=1:n], start=x0[i])
@variable(model, y[i=1:n], start=y0[i])
@objective(model, Min, R)
@NLconstraint(model, inside[i=1:n], x[i]^2 + y[i]^2 <= (R - r)^2)
@NLconstraint(model, distance[i=1:n-1,j=i+1:n],
                (x[i] - x[j])^2 + (y[i] - y[j])^2 >= 4r^2)
status = solve(model)

x = getvalue(x)
y = getvalue(y)
R = getvalue(R)
fx = getobjectivevalue(model)

println("status = $status")
println("x = $x")
println("y = $y")
println("R = $R")
println("f = $fx")

In [ ]:
t = linspace(0, 2π, 360)
plot(R * cos.(t), R * sin.(t), lw=2, c=:green, leg=false)
for i = 1:n
    plot!(r * cos.(t) + x[i], r * sin.(t) + y[i], c=:blue, ann=(x[i], y[i], text("$i")))
end
plot!()

### Posição de sensores

Um campo plano tem vários sensores, cada um consegue medir a posição de todos até uma distância $r$. Infelizmente esse medição tem um erro, que desconhecemos. Queremos saber a posição de todos esses sensores.

### Gerando dados falsos

In [ ]:
n = 100
minr = 0.5
srand(0)
x, y = rand(n), rand(n)
scatter(x, y, ms=3, leg=false, c=:black)
xlims!(-0.1, 1.1)
ylims!(-0.1, 1.1)

In [ ]:
scatter(x, y, ms=3, leg=false, c=:black)
t = linspace(0, 2π, 60)
for i = 1:n
    plot!(minr * cos.(t) + x[i], minr * sin.(t) + y[i], c=:green, l=:dot)
end
xlims!(-0.1, 1.1)
ylims!(-0.1, 1.1)

In [ ]:
noise = 1e-2
D = zeros(n, n)
for i = 1:n
    for j = i:n
        d = (x[i] - x[j])^2 + (y[i] - y[j])^2
        if d < minr
            D[i,j] = D[j,i] = d + noise * rand()
        end
    end
end

### Começo do modelo

A partir de agora, imaginamos que $D$ foi obtido pelas medições, representando o quadrado das distâncias entre os sensores $i$ e $j$. $d_{ij}$ nulo quer dizer que a distância é maior que a mensurável pelo equipamento.

A distância entre dois sensores $(X_i,Y_i)$, $(X_j,Y_j)$ deveria ser $\sqrt{D_{i,j}}$. Infelizmente, como temos um erro, não podemos esperar que isso aconteça. Então vamos justamente minimizar esse erro
$$ \min \sum_{j > i: D_{i,j} > 0} \bigg((X_i-X_j)^2 + (Y_i-Y_j)^2 - D_{i,j}\bigg)^2. $$

In [ ]:
model = Model(solver = IpoptSolver(print_level=0))
@variable(model, X[i=1:n], start=rand())
@variable(model, Y[j=1:n], start=rand())
@NLexpression(model, d[i=1:n,j=i+1:n], (X[i] - X[j])^2 + (Y[i] - Y[j])^2)
@NLobjective(model, Min, sum( (d[i,j] - D[i,j])^2 for i = 1:n, j = i+1:n if D[i,j] > 0))

status = solve(model)
X = getvalue(X)
Y = getvalue(Y)
println("f = $(getobjectivevalue(model))")

In [ ]:
scatter(X, Y, ms=3, leg=false, c=:black)

In [ ]:
scatter(x, y, ms=3, leg=false, c=:black)

**São os mesmos?**

In [ ]:
using Interact
μx, μy = mean(x), mean(y)
μX, μY = mean(X), mean(Y)
@manipulate for θ = linspace(0, 2π, 720), flip = [false, true]
    scatter(x - μx, y - μy, ms=3, leg=false, c=:black)
    RX = (X - μX) * cos(θ) - (Y - μY) * sin(θ)
    RY = (X - μX) * sin(θ) + (Y - μY) * cos(θ)
    if flip
        RY = -RY
    end
    scatter!(RX, RY, ms=3, leg=false, c=:red, m=(stroke(0)))
    xlims!(-0.8, 0.8)
    ylims!(-0.8, 0.8)
end

### Bard function

Um exemplo antigo de ajuste de dados não linear.

In [ ]:
y = [0.14; 0.18; 0.22; 0.25; 0.29;
     0.32; 0.35; 0.39; 0.37; 0.58;
     0.73; 0.96; 1.34; 2.10; 4.39]
t = 1:15
scatter(t, y, leg=false, c=:black, ms=3)

In [ ]:
@manipulate for a = linspace(0.01, 2, 100),
                b = linspace(1, 3, 100),
                c = linspace(1, 3, 100)
    m(t) = a + t / ((16 - t) * b + min(t, 16 - t) * c)
    scatter(t, y, leg=false, c=:black, ms=3)
    plot!(m, 1, 15, c=:red, lw=2)
end

In [ ]:
model = Model(solver = IpoptSolver(print_level=0))
u = collect(1:15)
v = 16 - u
w = min.(u, v)
@variable(model, x[1:3], start=1.0)
@NLobjective(model, Min, sum((x[1] + u[i] / (v[i] * x[2] + w[i] * x[3]) - y[i])^2 for i = 1:15))

status = solve(model)
x = getvalue(x)
println("x = $x")
println("f = $(getobjectivevalue(model))")

In [ ]:
scatter(t, y, leg=false, c=:black, ms=3)
plot!(t -> x[1] + t / ((16 - t) * x[2] + min(t, 16 - t) * x[3]), 1, 15, c=:red, lw=2)

### Máquina de Vetor Suporte

In [ ]:
m = 50
srand(0)
data = [randn(div(m,2))+1.5 randn(div(m,2))+1.5;
        randn(div(m,2))-1.5 randn(div(m,2))-1.5]
y = sign.(data[:,1] + data[:,2])
I = find(y .> 0)
scatter(data[I,1], data[I,2], m=:square, ms=5, c=:blue, leg=false)
I = find(y .< 0)
scatter!(data[I,1], data[I,2], m=:circle, ms=6, c=:red)

Linearmente separável: $w^Tx = b$, $w, x \in \mathbb{R}^n$, $b \in \mathbb{R}$, separa os dois conjuntos.

Se escolhermos $w$ e $b$ de modo que $w^Tx - b = \pm 1$ para os pontos mais próximos do conjunto, a margem será proporcional à $\frac{1}{\Vert w\Vert_2}$. Queremos maximizar a margem, então podemos minimizar $\Vert w\Vert_2$.
Cada ponto $x_i \in \mathbb{R}^n$ é classificado como $y_i = \pm 1$. Assim, podemos modelar o problema como.

$$ \min \frac{1}{2}\Vert w\Vert_2^2$$
$$ (w^Tx_i - b_i)y_i \geq 1. $$

In [ ]:
model = Model(solver = IpoptSolver(print_level=0))
@variable(model, w[1:2])
@variable(model, b)
@objective(model, Min, w[1]^2 + w[2]^2)
@constraint(model, [i=1:m], (w[1] * data[i,1] + w[2] * data[i,2] - b) * y[i] >= 1.0)

status = solve(model)
w = getvalue(w)
b = getvalue(b)

In [ ]:
I = find(y .> 0)
scatter(data[I,1], data[I,2], m=:square, ms=5, c=:blue, leg=false)
I = find(y .< 0)
scatter!(data[I,1], data[I,2], m=:circle, ms=6, c=:red)
plot!(t -> (b - w[1] * t) / w[2], extrema(data[:,1])..., lw=2, c=:black)
plot!(t -> (b + 1 - w[1] * t) / w[2], extrema(data[:,1])..., l=:dash, c=:black)
plot!(t -> (b - 1 - w[1] * t) / w[2], extrema(data[:,1])..., l=:dash, c=:black)

Na prática, raramente os dados são separáveis.

In [ ]:
m = 50
srand(0)
data = [randn(div(m,2))+1.5 randn(div(m,2))+1.5;
        randn(div(m,2))-1.5 randn(div(m,2))-1.5;
        randn(3,2)]
y = sign.(data[:,1] + data[:,2])
y[m+1:end] *= -1
m = length(y)
I = find(y .> 0)
scatter(data[I,1], data[I,2], m=:square, ms=5, c=:blue, leg=false)
I = find(y .< 0)
scatter!(data[I,1], data[I,2], m=:circle, ms=6, c=:red)

Como o conjunto não é mais linearmente separáveis, penalizamos o ponto classificado errado.

$$ \min \frac{1}{2}\Vert w\Vert_2^2 + C\sum_{i=1}^m\xi_i$$
$$ (w^Tx_i - b_i)y_i \geq 1 - \xi_i, \quad \xi \geq 0 $$

In [ ]:
C = 100.0
model = Model(solver = IpoptSolver(print_level=0))
@variable(model, w[1:2])
@variable(model, b)
@variable(model, ξ[1:m] >= 0)
@objective(model, Min, w[1]^2 + w[2]^2 + C * sum(ξ[i] for i = 1:m))
@constraint(model, [i=1:m], (w[1] * data[i,1] + w[2] * data[i,2] - b) * y[i] >= 1.0 - ξ[i])

status = solve(model)
w = getvalue(w)
b = getvalue(b)

I = find(y .> 0)
scatter(data[I,1], data[I,2], m=:square, ms=5, c=:blue, leg=false)
I = find(y .< 0)
scatter!(data[I,1], data[I,2], m=:circle, ms=6, c=:red)
plot!(t -> (b - w[1] * t) / w[2], extrema(data[:,1])..., lw=2, c=:black)
plot!(t -> (b + 1 - w[1] * t) / w[2], extrema(data[:,1])..., l=:dash, c=:black)
plot!(t -> (b - 1 - w[1] * t) / w[2], extrema(data[:,1])..., l=:dash, c=:black)

In [ ]:
# Exemplo do curso
# https://www.superdatascience.com/machine-learning/
# Part 3, section 16
using CSV, DataFrames
data = CSV.read("Social_Network_Ads.csv")
print(data)

In [ ]:
m = size(data, 1)
X = [get(data[i,j]) for i = 1:m, j = 3:4]
n = size(X, 2)
y = [get(data[i,5]) * 2 - 1 for i = 1:m]
P = find(y .> 0); N = find(y .< 0)
scatter(X[P,1], X[P,2], c=:blue, m=(:xcross,stroke(0)), ms=4, leg=false)
scatter!(X[N,1], X[N,2], c=:red, m=(:circle,stroke(0)), ms=3)
xlabel!("Idade")
ylabel!("Salário")

Vamos separar 80% dos dados acima para treinar nosso modelo, e 20% para testá-lo.

In [ ]:
srand(0)
M = round(Int, 0.8m) # 80% para testes
I = randperm(m)[1:M] 
J = setdiff(1:m, I)

scatter(X[intersect(P,I),1], X[intersect(P,I),2], c=:blue,
            m=(:xcross,stroke(0)), ms=4, leg=false)
scatter!(X[intersect(N,I),1], X[intersect(N,I),2], c=:red, m=(:circle,stroke(0)), ms=3)
xlabel!("Idade")
ylabel!("Salário")
title!("Treinamento")

In [ ]:
scatter(X[intersect(P,J),1], X[intersect(P,J),2], c=:blue,
            m=(:xcross,stroke(0)), ms=4, leg=false)
scatter!(X[intersect(N,J),1], X[intersect(N,J),2], c=:red, m=(:circle,stroke(0)), ms=3)
xlabel!("Idade")
ylabel!("Salário")
title!("Teste")

In [ ]:
C = 100.0
model = Model(solver = IpoptSolver(print_level=0))
@variable(model, w[1:n])
@variable(model, b)
@variable(model, ξ[1:M] >= 0)
@objective(model, Min, sum(w[j]^2 for j = 1:n) +
            C * sum(ξ[i] for i = 1:M))
@constraint(model, [i=1:M], (sum(w[j] * X[I[i],j] for j = 1:n) - b) * y[I[i]] >= 1.0 - ξ[i])

status = solve(model)
w = getvalue(w)
b = getvalue(b)

In [ ]:
K = intersect(find(y .> 0), I)
scatter(X[K,1], X[K,2], c=:blue, m=(:xcross,stroke(0)), ms=4, leg=false)
K = intersect(find(y .< 0), I)
scatter!(X[K,1], X[K,2], c=:red, m=(:circle,stroke(0)), ms=3)
plot!(t -> (b - w[1] * t) / w[2], extrema(X[:,1])..., lw=1, c=:black)
xlabel!("Idade")
ylabel!("Salário")
xlim = [minimum(X[:,1]), maximum(X[:,1])]
ylim = [minimum(X[:,2]), maximum(X[:,2])]
λ = 0.05 * [xlim[2] - xlim[1]; ylim[2] - ylim[1]]
xlims!(xlim[1] - λ[1], xlim[2] + λ[1])
ylims!(ylim[1] - λ[2], ylim[2] + λ[2])

In [ ]:
K = intersect(find(y .> 0), J)
scatter(X[K,1], X[K,2], c=:blue, m=(:xcross,stroke(0)), ms=4, leg=false)
K = intersect(find(y .< 0), J)
scatter!(X[K,1], X[K,2], c=:red, m=(:circle,stroke(0)), ms=3)
plot!(t -> (b - w[1] * t) / w[2], extrema(X[:,1])..., lw=1, c=:black)
xlabel!("Idade")
ylabel!("Salário")
xlim = [minimum(X[:,1]), maximum(X[:,1])]
ylim = [minimum(X[:,2]), maximum(X[:,2])]
λ = 0.05 * [xlim[2] - xlim[1]; ylim[2] - ylim[1]]
xlims!(xlim[1] - λ[1], xlim[2] + λ[1])
ylims!(ylim[1] - λ[2], ylim[2] + λ[2])

In [ ]:
# Matriz de confusão
#          | real -1 | real  1
# --------- --------- ---------
# pred -1  |    x    |    x
# pred  1  |    x    |    x
ytest = y[J]
ypred = sign.(X[J,:] * w - b)
[sum((ypred .== b1) .& (ytest .== b2)) for b1 = [-1,1], b2 = [-1,1]]

É fácil ver que usar uma reta nessa separação é uma escolha ruim. Também podemos usar SVM para outros tipos de separação, mas para isso precisamos ver o **dual** do problema. Não faremos isso aqui.